# IronLane Runtime Notebook
Minimal end-to-end loop: camera -> workers -> controller -> motors.


## Config
Core runtime settings and motor mapping.


In [ ]:
IMG = 224
CONTROL_HZ = 20.0

LEFT_GAIN = 0.11
ALLOW_REVERSE = False


## System Setup
Thread limits and OpenCV settings for stable runtime.


In [ ]:
!export CUDA_MODULE_LOADING=LAZY
import os

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

import cv2

cv2.setNumThreads(1)


## Controller
Steering + speed policy configuration.


In [ ]:
from ironlane.control import IronLaneController

ctl = IronLaneController(
    base_speed=0.16,
    slow_speed=0.12,
    stop_speed=0.0,
    steering_kp=0.09,
    steering_kd=0.40,
    steering_bias=0.0,
    steer_gain=0.40,
    lane_gain=0.20,
    lane_conf_min=0.15,
    steer_max_age_s=0.25,
    lane_max_age_s=0.25,
    yolo_max_age_s=0.60,
    steer_limit=0.20,
    avoid_speed_w=1.0,
)


## JetBot IO
Camera and motor interface.


In [ ]:
from jetbot import Camera, Robot, bgr8_to_jpeg

camera = Camera()
robot = Robot()


## Camera Callback
Grab the latest frame from the camera stream.


In [ ]:
latest_bgr = None


def _on_cam(change):
    global latest_bgr
    latest_bgr = change["new"]


camera.observe(_on_cam, names="value")


## Hubs and Workers
Shared hubs and inference workers.


In [ ]:
from ironlane.hub import FrameHub, ResultHub, StopFlag
from ironlane.modules import SteerPointWorker, SpeedMindWorker
from ironlane.yolo_policy import yolo_outputs_to_policy

framehub = FrameHub()
resulthub = ResultHub()
stop = StopFlag()


In [ ]:
steer_w = SteerPointWorker(
    framehub=framehub,
    resulthub=resulthub,
    stopflag=stop,
    engine_path="final_new2.plan",
    max_hz=8.0,
    mean=None,
    std=None,
)


yolo_w = SpeedMindWorker(
    framehub=framehub,
    resulthub=resulthub,
    stopflag=stop,
    engine_path="finalreport.engine",
    max_hz=4.0,
    postprocess_fn=lambda outs, ts=None, in_shape=None: yolo_outputs_to_policy(
        outs,
        ts=ts,
        in_shape=in_shape,
        conf_th=0.7,
        iou_th=0.5,
        alert_width=50.0,
        alert_width_by_cls={
            0: 50.0,
            1: 75.0,
            2: 38.0,
            3: 50.0,
            4: 50.0,
            5: 50.0,
        },
        pedestrian_slow_s=2.0,
        rail_stop_s=5.0,
        stop_sign_s=2.0,
        car_avoid_s=0.45,
        obstacle_avoid_s=0.45,
        car_avoid_strength=0.35,
        obstacle_avoid_strength=0.35,
        turn_sign=-1.0,  # Flip if steering direction is inverted.
    ),
)

import time

steer_w.start()

time.sleep(1)
yolo_w.start()


## Motor Helpers
Motor command scaling and safety clamp.


In [ ]:
from ironlane.utils import clip

def set_motors_signed(left, right):
    # left/right in [-1, 1]
    left = clip(float(left), -1.0, 1.0)
    right = clip(float(right), -1.0, 1.0)

    if not ALLOW_REVERSE:
        left = clip(left, 0.0, 1.0)
        right = clip(right, 0.0, 1.0)

    left = clip(left * (1.0 + LEFT_GAIN), -1.0, 1.0)
    robot.set_motors(left, right)


## Control Loop
Main loop: publish frame -> compute control -> drive motors.


In [ ]:
import time
import cv2
import ipywidgets
from IPython.display import display

dt = 1.0 / float(CONTROL_HZ)
last_print = time.time()

image_w = ipywidgets.Image(format="jpeg")
display(image_w)

try:
    while True:
        t0 = time.time()

        fbgr = latest_bgr
        if fbgr is None:
            time.sleep(0.01)
            continue

        fbgr_224 = cv2.resize(fbgr, (IMG, IMG), interpolation=cv2.INTER_AREA)
        frgb_224 = cv2.cvtColor(fbgr_224, cv2.COLOR_BGR2RGB)

        framehub.set(frgb_224, ts=t0)

        left, right, dbg = ctl.compute(resulthub)
        kind = dbg.get("yolo_hold_kind")

        set_motors_signed(left, right)

        image_w.value = bgr8_to_jpeg(fbgr_224)

        now = time.time()
        if now - last_print > 0.5:
            last_print = now
            print(f"kind: {kind}, ignore left {dbg.get('yolo_ignore_left', 0.0)}")

        elapsed = time.time() - t0
        if elapsed < dt:
            time.sleep(dt - elapsed)

except KeyboardInterrupt:
    pass
finally:
    camera.unobserve(_on_cam, names="value")
    camera.stop()
    robot.stop()
    stop.stop()
    framehub.close()
